In [ ]:
# imports
# -------

import numpy as np
import random
import copy
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import csv
import cv2
import h5py
import time
import scipy
from tqdm import tqdm
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import os
from os import listdir
from os.path import isfile, join
from PIL import Image
import pandas as pd
import multiprocessing
from scipy.ndimage import label
from skimage import measure
from scipy import interpolate
import json
import pickle
from skimage.transform import resize

# torch related imports
# ---------------------
import torch
from torch.autograd import Variable
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
import torchvision.utils as vutils
from torchvision.utils import save_image
import torch.optim as optim

# local settings
# --------------
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
    
# storage related imports
# -----------------------
import tempfile
from tempfile import TemporaryFile
from google.cloud import storage

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/home/venkateshmadhava/datasets/ven-ml-project-387fdf3f596f.json'

%matplotlib inline
%env JOBLIB_TEMP_FOLDER=/tmp


# Code

### Helper Functions

In [ ]:
# Function to check accuracy on regression model
# ----------------------------------------------

def check_accuracy(x_in,y_in,model,mode):
    
    
    # 0. sanity stuff
    # ---------------
    model = model.eval()
    m = x_in.shape[0]
    no_classes = y_in.shape[3]
    x_intrch = setup_image_tensor(x_in)
    
    # 0. predicting output
    # --------------------
    out = chunk_pass(x_intrch,model,False,use_cuda,1)
    out = out.view(out.size()[0],-1).cpu().data.numpy()
    
    # 1. accuracy ops
    # ---------------
    if mode == 'sigmoid':
        out[out >= 0.5] = 1
        out[out < 0.5] = 0
    else:
        out = softmax_to_onehot(out)
    
    sum_sim = np.sum((out == y_in.reshape(m,no_classes)), axis = 1)
    total_correct = np.sum((sum_sim == no_classes).astype('int'))  
    print('Total correct is : ' + str(total_correct) + ' (' + str(round((total_correct/m)*100,2))  + '%)')


    
#### simple function to convert softmax to one hot
# -------------------------------------------------
def softmax_to_onehot(p):
    
    # 0. converting p to numpy
    # ------------------------
    xout = np.zeros(p.shape)
    
    # 1. unfortuately havint to loop through
    # --------------------------------------
    inds = p.argmax(axis = 1)
    for i in range(inds.shape[0]):
        xout[i,inds[i]] = 1
    
    # 2. setting output to pytorch
    # ----------------------------
    return xout

In [ ]:
# getting file from google cloud storage
# ---------------------------------------

def get_file_from_google_storage(file_name):
    
    '''
    
    1. takes an input google cloud storage file name
    2. downloads to temp file
    3. returns local temp file path
    
    '''
    
    # 0. initialising bucket
    # ----------------------
    print('0. initialising bucket..')
    bucket_name = 'gpu_datatset_bucket'
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    
    # 1. retrieveing blob
    # -------------------
    print('1. retrieving blob..')
    blob = bucket.blob(file_name)
    
    # 2. downloading blob to temp file
    # --------------------------------
    print('2. downloading blob to temp file, this may take a while..')
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as gcs_tempfile:
        blob.download_to_filename(gcs_tempfile.name)
        
    # 3. final return
    # ---------------
    print('Done.')
    return gcs_tempfile.name

In [ ]:
# simple function to take in a folder and save images in a dict
# -------------------------------------------------------------


def create_dataset_from_folder_to_dict(infolder):
    
    # 0. initialisations
    # ------------------
    d = {}
    image_list = [f for f in listdir(infolder) if isfile(join(infolder, f)) and '.jpg' in f.lower()]
    assert len(image_list) > 0, 'No images found in the folder'
    assert len(image_list) <= 100, 'More than 100 images found. Cannot proceed further using for loop.'
    
    # 1. simple iteration
    # -------------------
    for i in range(len(image_list)):
        print('At image ' + str(i+1) + ' of ' + str(len(image_list)) + ' images..', end = '\r')
        img = cv2.imread(join(infolder, image_list[i]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        d[i] = img
    
    # 2. final return
    # ---------------
    print('\ndone.')
    return d
 

In [ ]:
# Pool function to create a dataset from input folder
# ---------------------------------------------------

def create_dataset_from_folder_all(infolder,n_h,n_w):
    
    # 0. initialisations
    # ------------------
    image_list = [f for f in listdir(infolder) if isfile(join(infolder, f)) and '.jpg' in f.lower()]
    assert len(image_list) > 0, 'No images found in the folder'
    xout = np.zeros((len(image_list),n_h,n_w,3), dtype='uint8')
    
    # 1. building args
    # ----------------
    all_args = []
    for i in range(xout.shape[0]):
        all_args.append((i,xout,infolder,image_list,n_h,n_w))
    
    # 2. calling resize function across multiprocessing pool
    # ------------------------------------------------------
    pool = ThreadPool(5)
    pool.starmap(create_dataset_from_folder_single, all_args)
    print('Done creating a dataset with ' + str(xout.shape[0]) + ' images.')
    
    return xout

    
# FUNCTION 2
# GENERIC FUNCTION - to resize a single image
# ------------------------------------------
def create_dataset_from_folder_single(i,xout,infolder,image_list,n_h,n_w):
    
    # snippet
    # -------
    name = image_list[i]
    img = cv2.imread(join(infolder, name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (n_w,n_h))
    xout[i] = img



In [ ]:
# Pool function to resize images
# ------------------------------

def resize_all(ximgs,new_h,new_w,nearest_interpolate=False):
    
    # 0. initialisations
    # ------------------
    c = ximgs.shape[3]
    x_images_resized = np.zeros((ximgs.shape[0],new_h,new_w,c), dtype='uint8')
    
    # 1. building args
    # ----------------
    all_args = []
    for i in range(ximgs.shape[0]):
        all_args.append((i,ximgs[i],x_images_resized,new_h,new_w,nearest_interpolate))
    
    # 2. calling resize function across multiprocessing pool
    # ------------------------------------------------------
    pool = ThreadPool(5)
    pool.starmap(resize_image_single, all_args)
    print('Done resizing ' + str(ximgs.shape[0]) + ' images.')
    
    return x_images_resized

    
# FUNCTION 2
# GENERIC FUNCTION - to resize a single image
# ------------------------------------------
def resize_image_single(i,x_in,x_out,new_h,new_w,nearest_interpolate_mode):
    
    # simple code
    # -----------
    img = x_in
    if nearest_interpolate_mode == True:
        img = cv2.resize(img, (new_w,new_h), interpolation=cv2.INTER_NEAREST)
    else:
        img = cv2.resize(img, (new_w,new_h))
    x_out[i] = img
   

In [ ]:
# SIMPLE FUNTION TO CONVERT RGB TO GRAYSCALE
# -------------------------------------------
def rgb2gray(x):
    

    x[:,:,:,0] = x[:,:,:,0] * 0.2989
    x[:,:,:,1] = x[:,:,:,0] * 0.5870
    x[:,:,:,2] = x[:,:,:,0] * 0.1140
    xout = np.sum(x,axis = 3)
    

    #r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    #gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return xout

In [ ]:
# unpicle cifar 10
# ----------------

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


In [ ]:
# simple snippet to find out the distribution of class values
# -----------------------------------------------------------

def return_class_distribution(yin):
    
    '''
    
    1. returns a simple dict with number of examples in each class in y
    2. input must be of shape (m,c)
    3. returns d[class_number] = num_examples
    
    '''
    
    # 0. initialisations
    # ------------------
    m,c = yin.shape
    d = {}
    
    # 1. looping through classes
    # --------------------------
    for curr_c in range(c):
        d[curr_c] = np.argwhere(yin[:,curr_c] == 1).shape[0]
    
    # 2. printing results
    # -------------------
    for keys in d:
        print('class ' + str(keys) + ': ' + str(d[keys]))
    
    # 3. final return
    # ---------------
    return d
        
    
    
    

In [ ]:
# main function to load cifar 10 batches into numpy arrays, threaded function
# ---------------------------------------------------------------------------

def load_cifar10(b_url):
    
    ''' loads a batch of cifar 10 images, returns x,y '''
    
    # 0. initialisations
    # ------------------
    global b_dict
    b_dict = unpickle(b_url)
    m = b_dict[b'data'].shape[0]
    no_classes = 10
    h,w,c = 32,32,3
    
    # 0.1 some global initialisations as well
    # ---------------------------------------
    global x
    x = np.zeros((m,h,w,c), dtype = 'uint8')
    global y
    y = np.zeros((m,no_classes))
    
    # 1. threaded function
    # --------------------
    pool = ThreadPool(5)
    pool.map(load_cifar10_single, list(range(m)))
    print('Done with ' + str(m) + ' images. Access at global x and global y.')
    
    
    
# cifar 10 single function
# ------------------------
def load_cifar10_single(i):
    
    
    # 0. getting in global variables
    # ------------------------------
    global b_dict
    global x
    global y
    
    # 1. setting indices
    # ------------------
    len_r = 1024
    r_end = len_r
    g_str = r_end
    g_end = g_str + len_r
    b_str = g_end
    b_end = b_str + len_r
    r_end,g_str,g_end,b_str,b_end
    
    # 2. setting channel values
    # -------------------------
    img_row = b_dict[b'data'][i]
    r_channel = img_row[0:r_end].reshape(32,32,1)
    g_channel = img_row[g_str:g_end].reshape(32,32,1)
    b_channel = img_row[b_str:b_end].reshape(32,32,1)
    im = np.concatenate((r_channel,g_channel,b_channel), axis = 2)
    
    # 3. final ops
    # ------------
    x[i] = im
    y[i,b_dict[b'labels'][i]] = 1


# NN related code

In [ ]:
# a simple function to generate output from latent
# ------------------------------------------------

def generate_output(xin,model,start_ind,end_ind,print_images,use_cuda):
    
    
    # 1. generating output
    # --------------------
    xout = chunk_pass(xin[start_ind:end_ind],model.eval(),False,use_cuda,1)
    
    # images dataset
    # --------------
    xout_gen = to_numpy_image(xout.cpu().data)#.astype('uint8')
    xout_orig = to_numpy_image(xin[start_ind:end_ind].cpu().data).astype('uint8')
        
 
    # 4. priniting images
    # -------------------
    if print_images == True:
        for i in range(xout_orig.shape[0]):
            print('Example ' + str(i) + '..')
            print('----------------------')
            print('Original - ')
            plt.figure(figsize=(5,5))
            plt.imshow(xout_orig[i])
            plt.show()
            print('Generated - ')
            plt.figure(figsize=(5,5))
            plt.imshow(xout_gen[i])
            plt.show()
            print('\n----------------\n')
    
    # returns
    # -------
    return xout_orig, xout_gen


In [ ]:
# Super helpful chunker function that returns seq chunks correctly sized even at ends
# -----------------------------------------------------------------------------------
# GENERIC function to calculate conv outsize
# ------------------------------------------
def outsize_conv(n_H,n_W,f,s,pad):
    
    h = ((n_H - f + (2*pad))/s) + 1
    w = ((n_W - f + (2*pad))/s) + 1
    return h,w
    
    
# GENERIC function to calculate upconv outsize
# --------------------------------------------    
def outsize_upconv(h,w,f,s,p):
    hout = (h-1)*s - 2*p + f
    wout = (w-1)*s - 2*p + f
    return hout, wout



def chunker(seq, size):
    
    # from http://stackoverflow.com/a/434328
    # not touch this code
    # -------------------
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))



# GENERIC - initialises weights for a NN
# --------------------------------------
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
    #    print(classname)
        m.weight.data.normal_(0.0, 0.02)
    #elif classname.find('Linear') != -1:
    #    print(classname)
    #    m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
        
# GENERIC - change an torch image to numoy image
# ----------------------------------------------
def to_numpy_image(xin):
    
    try:
        xin = xin.data.numpy()
    except:
        xin = xin.numpy()
    
    
    xout = np.swapaxes(xin,1,2)
    xout = np.swapaxes(xout,2,3)
    
    # returns axes swapped numpy images
    # ---------------------------------
    return xout       



# GENERIC - converts numpy images to torch tensors for training
# -------------------------------------------------------------
def setup_image_tensor(xin):
    xout = np.swapaxes(xin,1,3)
    xout = np.swapaxes(xout,2,3)
    
    # returns axes swapped torch tensor
    # ---------------------------------
    xout = torch.from_numpy(xout)
    return xout.float()


# A functino to get linemarkings
# -------------------------------

def get_ae_output_image(x,net,use_cuda):
    
    # 0. Setting up input as torch
    # ----------------------------
    x_t = Variable(setup_image_tensor(x)).float()
        
        
    # 1. Using chunk pass to get linemarkings
    # ----------------------------------------
    xout = chunk_pass(x_t,net.eval(),False,use_cuda,1)
    xout = to_numpy_image(xout.cpu().data)
    xout = (xout * 255).astype('uint8')
    
    # 2. final return
    # ---------------
    return xout
    
    
# GENERIC class that inherits nn module and makes a sequential object a model
# ---------------------------------------------------------------------------
class Net(nn.Module):
    def __init__(self,sequencelist):
        super().__init__() # Initializing nn.Module construtors
        self.forwardpass = sequencelist
        
    def forward(self,x):
        xout = self.forwardpass(x)
        return xout

In [ ]:
# Function to build a lineaf FC model
# -----------------------------------

def linear_fc(layers, nw_activations, target_activation, dropout_p):
    
    'The first value in the layers list is the input dimensions of the input'
    
    # 0. initialisations
    # ------------------
    seq_list = []
    
    # setting N/W activations
    # -------------------
    if nw_activations == 'relu':
        nw_act = nn.ReLU()
    elif nw_activations == 'lrelu':
        nw_act = nn.LeakyReLU(0.2)
    elif nw_activations == 'sigmoid':
        nw_act = nn.Sigmoid()
    elif nw_activations == 'tanh':
        nw_act = nn.Tanh()
    else:
        nw_act = nn.ReLU()
    
    # setting target activations
    # --------------------------
    if target_activation == 'sigmoid':
        target_act = nn.Sigmoid()
    elif target_activation == 'softmax':
        target_act = nn.Softmax()
    else:
        target_act = None
    
    # 1. building n/w's layer list
    # ----------------------------
    network = []
    for k in range(len(layers)):
        try:
            network.append((layers[k],layers[k+1]))
        except:
            pass
            

    
    # 2. constructing encoder n/w
    # ----------------------------
    for i in range(len(network)):
        
        # 2.1 adding linear layers to encoder
        # ------------------------------------
        curr_dims = network[i]
        seq_mod = nn.Linear(curr_dims[0],curr_dims[1])
        seq_list.append(seq_mod)
        
        # checking last layer or not
        # --------------------------
        if i+1 == len(network):
            
            # at last layer
            # -------------
            if target_act == None:
                pass
            else:
                seq_list.append(target_act)
        
        else:
            
            # batchnorm
            # ---------
            seq_list.append(nn.BatchNorm1d(curr_dims[1]))
          
            # non linear activation
            # ---------------------
            seq_list.append(nw_act)
            
            # dropout
            # -------
            seq_list.append(nn.Dropout(p = dropout_p))
           
            
    
    # 3. returning model
    # ------------------
    seq_list = nn.Sequential(*seq_list)
    seq_list.apply(weights_init)

    model = Net(seq_list)
    model = model.train()
    
    return model
            
            


In [ ]:
# GENERIC model function to train the networks
# --------------------------------------------

def model_train(xin,yin,xval,yval,load_mode,model,epochs,mbsize,loss_mode,use_cuda,save_state,path):
    
    # 0. initialisations
    # ------------------
    loss_train = []
    loss_val = []
    norm_flag = 0

    

    # normalising input to 0-1 while making sure this is an image
    # -----------------------------------------------------------
    if len(xin.size()) > 3:
            
        # if the input and output are images - try will go through with the statement
        # ----------------------------------------------------------------------------
        if len(xin.size()) > 3 and len(yin.size()) > 3:

            assert torch.mean(xin).item() > 1 and torch.mean(yin).item() > 1, 'Input data is already in range 0-1. Not consistent with flow.'
            

            # both need to be normalised
            # --------------------------
            #xin = xin/255
            #yin = yin/255
            norm_flag = 1
            print('Input and Output dataset will be normalised to 0-1')

        
        # incase input and output both are NOT images
        # -------------------------------------------
        else:

            assert torch.mean(xin).item() > 1, 'Input data is already in range 0-1. Not consistent with flow.'

            # normalising input
            # -----------------
            #xin = xin/255
            norm_flag = 2
            print('Input dataset will be normalised to 0-1')
            
            
    
    # ensuring xval and yval are None
    # -------------------------------
    assert xval == None and yval == None, 'xval and yval provided, but there is no code to normalise'
    
    
    if load_mode == 'from saved':
        
        # loading from saved
        # ------------------
        model,optimizer,saved_epoch,saved_loss,saved_loss_mode = load_saved_model_function(path,use_cuda)
        model = model.train()
        loss_mode = saved_loss_mode
        print('Loading model from saved state...')
        print('Last saved loss - ' + str(saved_loss))
        print('Last saved epoch - ' + str(saved_epoch))
        epochs += int(saved_epoch)
        start_epoch = int(saved_epoch)
        
    else:
        
        # building new
        # ------------
        start_epoch = 1
        model = model.train()
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()))
        #optimizer = torch.optim.Adadelta(filter(lambda p: p.requires_grad,model.parameters()))
        #optimizer = torch.optim.RMSprop(filter(lambda p: p.requires_grad,model.parameters()))
        #optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad,model.parameters()), lr=0.1, momentum=0.9)
        
        
    
    
    # model set up as per cuda
    # ------------------------
    if use_cuda == True:
        torch.cuda.empty_cache()
        model = model.cuda()        
    
    
    # setting loss criterion
    # ----------------------
    if loss_mode == 'MSE':
        criterion = nn.MSELoss()
    elif loss_mode == 'BCE':
        criterion = nn.BCELoss()
    elif loss_mode == 'NLL':
        criterion = nn.NLLLoss()
    elif loss_mode == 'crossentropy':
        criterion = nn.CrossEntropyLoss()
        yin = torch.max(yin.long(),1)[1]
    else:
        criterion = nn.MSELoss()
        
    
    # 1. Setting up minibatch features
    # --------------------------------
    m = xin.size()[0]
    mb_list = []
    mb_list = list(range(int(m/mbsize)))
    if m % mbsize == 0: # if the minibatches can be split up perfectly.
        'do nothing'
    else:
        mb_list.append(mb_list[len(mb_list)-1] + 1)
        
    # 2. Actual iters
    # ----------------
    for i in range(start_epoch,epochs+1):
            
        for p in mb_list:
            
            # Mini batch operations
            # ---------------------
            start_index = p*mbsize
            end_index = m if p == mb_list[len(mb_list)-1] else p*mbsize + mbsize
            m_curr = end_index - start_index
            
            Xin_mb = xin[start_index:end_index]
            Yin_mb = yin[start_index:end_index]
            
            if use_cuda == True:
                Xin_mb = Xin_mb.cuda()
                Yin_mb = Yin_mb.cuda()
                
            # normalising ops
            # --------------
            if norm_flag == 1:
                
                # normalise both input and target
                # -------------------------------
                Xin_mb = copy.deepcopy(Xin_mb)/255
                Yin_mb = copy.deepcopy(Yin_mb)/255
            
            else:
                
                # normalise only input
                # --------------------
                Xin_mb = copy.deepcopy(Xin_mb)/255
                
                
            # Network ops
            # -----------
            model_out = model(Xin_mb)
            optimizer.zero_grad()
            loss = criterion(model_out, Yin_mb) # loss(output, target)
            loss.backward()
            optimizer.step()
            loss_train.append(loss.item())
            
            # deleting curr variables
            # -----------------------
            if use_cuda == True:
                Xin_mb = Xin_mb.cpu()
                Yin_mb = Yin_mb.cpu()
                model_out = model_out.cpu()
                
                del Xin_mb
                del Yin_mb
                del model_out
                torch.cuda.empty_cache()
            
            # printing loss
            # -------------
            print('Epoch ' + str(i) + ', minibatch ' + str(p+1) + ' of '  +  str(len(mb_list)) + ' -- Model loss: ' + str(loss.item()))
            

    # 3. outside for loop saving model state
    # --------------------------------------
    if save_state == True and epochs+1 > start_epoch:
        
        # 3.1 initialising save dict
        # --------------------------
        save_dict = {}
        save_dict['epoch'] = str(i)
        save_dict['model_state_dict'] = model.cpu().state_dict()
        save_dict['optimizer_state_dict'] = optimizer.state_dict()
        save_dict['loss'] = str(loss.cpu().item())
        save_dict['loss_mode'] = loss_mode
        
        
        # 3.2 saving
        # ----------
        torch.save(save_dict,path)
        
        # saving full model to initialise a new model later on
        # ----------------------------------------------------
        torch.save(model.cpu(),path.replace('.tar','_MODEL.tar'))
        
        print('Saved.')
        
    
    # 4. return model in order to use elsewhere in the code
    # -----------------------------------------------------
    return model
        


In [ ]:
# a function to load a saved model
# --------------------------------

def load_saved_model_function(path, use_cuda):
    
    
    ''' path = /folder1/folder2/model_ae.tar format'''
    
    # 1. loading full model
    # ---------------------
    model = torch.load(path.replace('.tar','_MODEL.tar'))
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad,model.parameters()))
    
    # 2. Applying state dict
    # ----------------------
    if use_cuda == True:
        
        # loads to GPU
        # ------------
        checkpoint = torch.load(path)
        
    else:
        # loads to CPU
        # ------------
        checkpoint = torch.load(path, map_location=lambda storage, loc: storage)
        
        
    # loading checkpoint
    # -------------------
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # loading optimizer
    # -----------------
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if use_cuda == True:
        for state in optimizer.state.values():
            for k, v in state.items():
                if isinstance(v, torch.Tensor):
                    state[k] = v.cuda()
            
            
            
    # loading other stuff
    # -------------------
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    loss_mode = checkpoint['loss_mode']
    
    return model, optimizer, epoch, loss, loss_mode
    
    


In [ ]:
# simple function to do a forward pass by chunks
# ----------------------------------------------

def chunk_pass(xin,model,latent,use_cuda,chunksize):
    
    # 0. some initialisations
    # -----------------------
    model = model.eval()
    if use_cuda == True:
        torch.cuda.empty_cache()
        model = model.cuda()
        
    
    # sanity assertion
    # ----------------
    if len(xin.size()) > 3:
        
        # normalising data
        # ----------------
        assert torch.mean(xin).item() > 1, 'Input data is already in range 0-1. Not consistent with flow.'
        xin = copy.deepcopy(xin)/255
        print("Normalised data to 0-1")
       

    # 1. chuck loop
    # -------------
    with tqdm(total=xin.size()[0]) as pbar:
        for i,chunk_data in enumerate(chunker(xin, chunksize)):
            
            # forward pass ops
            # ----------------
            try:
                chunk_data = Variable(chunk_data.data, volatile=True)
            except:
                chunk_data = Variable(chunk_data, volatile=True)
                
            if use_cuda == True:
                
                torch.cuda.empty_cache()
               
                if latent == True:
                    try:
                        curr_forwardpass = model.latent(chunk_data.cuda().detach())
                    except:
                        curr_forwardpass = model.latent(chunk_data.cuda())
                else:
                    try:
                        curr_forwardpass = model(chunk_data.cuda().detach())
                    except:
                        curr_forwardpass = model(chunk_data.cuda())
            else:
                
                if latent == True:
                    try:
                        curr_forwardpass = model.latent(chunk_data.cpu().detach())
                    except:
                        curr_forwardpass = model.latent(chunk_data.cpu())
                else:
                    try:
                        curr_forwardpass = model(chunk_data.cpu().detach())
                    except:
                        curr_forwardpass = model(chunk_data.cpu())
                
            # concat ops
            # ----------
            try:
                xout = torch.cat((xout,curr_forwardpass), 0)
            except:
                xout= curr_forwardpass
                
            # for memory purpose
            # ------------------
            if use_cuda == True:
                curr_forwardpass = curr_forwardpass.cpu()
                chunk_data = chunk_data.cpu()
                del curr_forwardpass
                del chunk_data
                torch.cuda.empty_cache()
            
            pbar.update(chunksize)
        
    # 2. return
    # ---------
    xout = Variable(xout.data, volatile=False).cpu()
    

    return xout
    
    

In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_ae_4_layer(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        added_act = nn.Tanh()
        dropout_prob = 0.1
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        #  what worked - 3,64,128,256,128,64,3
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(3,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 64
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 128
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nn.Softmax2d() #nw_activation_conv
        cl3 = [ct3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        #conv4 = 256
        #ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        #cb4 = nn.BatchNorm2d(conv4)
        #ca4 = nn.Softmax2d() #nw_activation_conv
        #cl4 = [ct4,ca4,dropout_node]
        #self.convl4 = nn.Sequential(*cl4) # size 6 x 4
        
        
        # Pooling layer
        mxpl =  [nn.MaxPool2d((3,3), stride=3)]
        #avpl =  [nn.AvgPool2d((6,4), stride=1)]
        self.pool_net = nn.Sequential(*mxpl)
        
        # Adding a fully connected linear layer
        #
        
        # Upconv layer 0
        ###
        #t0 = nn.ConvTranspose2d(conv4,conv4,2,stride = 2)
        #b0 = nn.BatchNorm2d(conv4)
        #a0 = nw_activation_conv
        #l0 = [t0,b0,a0]
        #self.upcl0 = nn.Sequential(*l0)
        
        # Upconv layer 1
        ###
        up_conv1 = 128
        t1 = nn.ConvTranspose2d(conv3,up_conv1,3,stride = 3)
        b1 = nn.BatchNorm2d(up_conv1)
        a1 = nw_activation_conv
        l1 = [t1,b1,a1,dropout_node]
        self.upcl1 = nn.Sequential(*l1)
        
        # Upconv layer 2
        ###
        up_conv2 = 64
        t2 = nn.ConvTranspose2d(up_conv1,up_conv2,f,stride = s)
        b2 = nn.BatchNorm2d(up_conv2)
        a2 = nw_activation_conv
        l2 = [t2,b2,a2,dropout_node]
        self.upcl2 = nn.Sequential(*l2)
        
        # Upconv layer 3
        ###
        up_conv3 = 32
        t3 = nn.ConvTranspose2d(up_conv2,up_conv3,f,stride = s)
        b3 = nn.BatchNorm2d(up_conv3)
        a3 = nw_activation_conv
        l3 = [t3,b3,a3,dropout_node]
        self.upcl3 = nn.Sequential(*l3)
        
        # Upconv layer 4
        ###
        t4 = nn.ConvTranspose2d(up_conv3,3,f,stride = s)
        a4 = nn.Sigmoid()
        l4 = [t4,a4]
        self.upcl4 = nn.Sequential(*l4)
        

    def forward(self, x):
        
        # Generation
        # ----------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        #c4_out = self.convl4(c3_out)
        c5_out = self.pool_net(c3_out)
        
        #f1_out = self.upcl0(c5_out)
        f2_out = self.upcl1(c5_out)
        f3_out = self.upcl2(f2_out)
        f4_out = self.upcl3(f3_out)
        f5_out = self.upcl4(f4_out)
        
        return f5_out

    
    def latent(self, x):
        
        
        # 0. forward prop
        # ---------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        

        # 1. Working out layer number
        # ---------------------------
        if self.layer_mode == 'deep':
            forward_out = self.convl4(c3_out)
        
        elif self.layer_mode == 'deep_minus_2':
            forward_out = c2_out
        
        else:
            forward_out = c3_out


        # 2. Including a pool layer - setting dims
        # ----------------------------------------
        if self.layer_dims_set == True:
            
            ih, iw, pool_stride = self.layer_f, self.layer_f, self.layer_s
            
        else:
            
            ih,iw = forward_out.size()[2],forward_out.size()[3]
            pool_stride = 1
        
        if self.pool_mode == 'avg':

            # avg pool - comment/uncomment
            # ----------------------------
            avpl =  nn.Sequential(*[nn.AvgPool2d((ih,iw), stride=pool_stride)])
            latent_out = avpl(forward_out)
        
        elif self.pool_mode == 'max':
            
            # maxpool - comment/uncomment
            # ---------------------------
            mxpl =  nn.Sequential(*[nn.MaxPool2d((ih,iw), stride=pool_stride)])
            latent_out = mxpl(forward_out)
            
        elif self.pool_mode == 'both':
            
            # avg pool
            # --------
            avpl =  nn.Sequential(*[nn.AvgPool2d((ih,iw), stride=pool_stride)])
            latent_out_avg = avpl(forward_out)
            latent_out_avg = latent_out_avg.view(latent_out_avg.size()[0],-1)
            
            # maxpool
            # -------
            mxpl =  nn.Sequential(*[nn.MaxPool2d((ih,iw), stride=pool_stride)])
            latent_out_max = mxpl(forward_out)
            latent_out_max = latent_out_max.view(latent_out_max.size()[0],-1)
            
            # final concat
            # ------------
            latent_out = torch.cat((latent_out_avg, latent_out_max), 1)
            
        
        else:
            
            # no pooling
            # ----------
            latent_out = forward_out
            
        
        return latent_out.view(latent_out.size()[0],-1)
        
    
     
    def set_pool_mode(self, pool_mode, layer_mode, layer_dims_set, layer_f, layer_s):
        
        # setting pool mode
        # -----------------
        self.pool_mode = pool_mode
        self.layer_mode = layer_mode
        self.layer_dims_set = layer_dims_set
        self.layer_f = layer_f
        self.layer_s = layer_s
        
        print('Modes set.')


In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_ae_3_layer(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        added_act = nn.Tanh()
        dropout_prob = 0.1
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        #  what worked - 3,64,128,256,128,64,3
        
        # Conv 1
        ###
        conv1 = 64
        ct1 = nn.Conv2d(3,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 128
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 256
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nn.Softmax2d() #nn.Softmax2d() #nw_activation_conv
        cl3 = [ct3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        #conv4 = 256
        #ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        #cb4 = nn.BatchNorm2d(conv4)
        #ca4 = nn.Softmax2d() #nw_activation_conv
        #cl4 = [ct4,ca4,dropout_node]
        #self.convl4 = nn.Sequential(*cl4) # size 6 x 4
        
        
        # Pooling layer
        #mxpl =  [nn.MaxPool2d((2,2), stride=2)]
        #avpl =  [nn.AvgPool2d((6,4), stride=1)]
        #self.pool_net = nn.Sequential(*mxpl)
        
        # Adding a fully connected linear layer
        #
        
        # Upconv layer 0
        ###
        #t0 = nn.ConvTranspose2d(conv4,conv4,2,stride = 2)
        #b0 = nn.BatchNorm2d(conv4)
        #a0 = nw_activation_conv
        #l0 = [t0,b0,a0]
        #self.upcl0 = nn.Sequential(*l0)
        
        # Upconv layer 1
        ###
        #up_conv1 = 128
        #t1 = nn.ConvTranspose2d(conv4,up_conv1,f,stride = s)
        #b1 = nn.BatchNorm2d(up_conv1)
        #a1 = nw_activation_conv
        #l1 = [t1,b1,a1,dropout_node]
        #self.upcl1 = nn.Sequential(*l1)
        
        # Upconv layer 2
        ###
        up_conv2 = 128
        t2 = nn.ConvTranspose2d(conv3,up_conv2,f,stride = s)
        b2 = nn.BatchNorm2d(up_conv2)
        a2 = nw_activation_conv
        l2 = [t2,b2,a2,dropout_node]
        self.upcl2 = nn.Sequential(*l2)
        
        # Upconv layer 3
        ###
        up_conv3 = 64
        t3 = nn.ConvTranspose2d(up_conv2,up_conv3,f,stride = s)
        b3 = nn.BatchNorm2d(up_conv3)
        a3 = nw_activation_conv
        l3 = [t3,b3,a3,dropout_node]
        self.upcl3 = nn.Sequential(*l3)
        
        # Upconv layer 4
        ###
        t4 = nn.ConvTranspose2d(up_conv3,3,f,stride = s)
        a4 = nn.Sigmoid()
        l4 = [t4,a4]
        self.upcl4 = nn.Sequential(*l4)
        

    def forward(self, x):
        
        # Generation
        # ----------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        #c4_out = self.convl4(c3_out)
        #c5_out = self.pool_net(c3_out)
        
        #f1_out = self.upcl0(c5_out)
        #f2_out = self.upcl1(c4_out)
        f3_out = self.upcl2(c3_out)
        f4_out = self.upcl3(f3_out)
        f5_out = self.upcl4(f4_out)
        
        return f5_out

    
    def latent(self, x):
        
        
        # 0. forward prop
        # ---------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        
        # 1. Working out layer number
        # ---------------------------
        if self.layer_mode == 'deep_minus_1':
            forward_out = c2_out
        
        else:
            forward_out = self.convl3(c2_out)
        

        # 2. Including a pool layer - setting dims
        # ----------------------------------------
        if self.layer_dims_set == True:
            
            ih, iw, pool_stride = self.layer_f, self.layer_f, self.layer_s
            
        else:
            
            ih,iw = forward_out.size()[2],forward_out.size()[3]
            pool_stride = 1
        
        if self.pool_mode == 'avg':

            # avg pool - comment/uncomment
            # ----------------------------
            avpl =  nn.Sequential(*[nn.AvgPool2d((ih,iw), stride=pool_stride)])
            latent_out = avpl(forward_out)
        
        elif self.pool_mode == 'max':
            
            # maxpool - comment/uncomment
            # ---------------------------
            mxpl =  nn.Sequential(*[nn.MaxPool2d((ih,iw), stride=pool_stride)])
            latent_out = mxpl(forward_out)
            
        elif self.pool_mode == 'both':
            
            # avg pool
            # --------
            avpl =  nn.Sequential(*[nn.AvgPool2d((ih,iw), stride=pool_stride)])
            latent_out_avg = avpl(forward_out)
            latent_out_avg = latent_out_avg.view(latent_out_avg.size()[0],-1)
            
            # maxpool
            # -------
            mxpl =  nn.Sequential(*[nn.MaxPool2d((ih,iw), stride=pool_stride)])
            latent_out_max = mxpl(forward_out)
            latent_out_max = latent_out_max.view(latent_out_max.size()[0],-1)
            
            # final concat
            # ------------
            latent_out = torch.cat((latent_out_avg, latent_out_max), 1)
            
        
        else:
            
            # no pooling
            # ----------
            latent_out = forward_out
            
        
        return latent_out.view(latent_out.size()[0],-1)
        
    
     
    def set_pool_mode(self, pool_mode, layer_mode, layer_dims_set, layer_f, layer_s):
        
        # setting pool mode
        # -----------------
        self.pool_mode = pool_mode
        self.layer_mode = layer_mode
        self.layer_dims_set = layer_dims_set
        self.layer_f = layer_f
        self.layer_s = layer_s
        
        print('Modes set.')


In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_ae_deep(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        added_act = nn.Tanh()
        dropout_prob = 0.2
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        #  what worked - 3,64,128,256,128,64,3
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(3,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 64
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 128
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv #nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        conv4 = 256
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) 
        
        # Conv 5
        ###
        conv5 = 512
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nn.Softmax2d()
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) 
        
        
        # Pooling layer
        #mxpl =  [nn.MaxPool2d((2,2), stride=2)]
        #avpl =  [nn.AvgPool2d((6,4), stride=1)]
        #self.pool_net = nn.Sequential(*mxpl)
        
        # Adding a fully connected linear layer
        #
        
        # Upconv layer 0
        ###
        up_conv0 = 256
        t0 = nn.ConvTranspose2d(conv5,up_conv0,f,stride = s)
        b0 = nn.BatchNorm2d(up_conv0)
        a0 = nw_activation_conv
        l0 = [t0,b0,a0,dropout_node]
        self.upcl0 = nn.Sequential(*l0)
        
        # Upconv layer 1
        ###
        up_conv1 = 128
        t1 = nn.ConvTranspose2d(up_conv0,up_conv1,f,stride = s)
        b1 = nn.BatchNorm2d(up_conv1)
        a1 = nw_activation_conv
        l1 = [t1,b1,a1,dropout_node]
        self.upcl1 = nn.Sequential(*l1)
        
        # Upconv layer 2
        ###
        up_conv2 = 64
        t2 = nn.ConvTranspose2d(up_conv1,up_conv2,f,stride = s)
        b2 = nn.BatchNorm2d(up_conv2)
        a2 = nw_activation_conv
        l2 = [t2,b2,a2,dropout_node]
        self.upcl2 = nn.Sequential(*l2)
        
        # Upconv layer 3
        ###
        up_conv3 = 32
        t3 = nn.ConvTranspose2d(up_conv2,up_conv3,f,stride = s)
        b3 = nn.BatchNorm2d(up_conv3)
        a3 = nw_activation_conv
        l3 = [t3,b3,a3,dropout_node]
        self.upcl3 = nn.Sequential(*l3)
        
        # Upconv layer 4
        ###
        t4 = nn.ConvTranspose2d(up_conv3,3,f,stride = s)
        a4 = nn.Sigmoid()
        l4 = [t4,a4]
        self.upcl4 = nn.Sequential(*l4)
        

    
    def forward(self, x):
        
        
        # forward pass
        # ------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        
        f1_out = self.upcl0(c5_out)
        f2_out = self.upcl1(f1_out)
        f3_out = self.upcl2(f2_out)
        f4_out = self.upcl3(f3_out)
        f5_out = self.upcl4(f4_out)
        
        return f5_out

    
    
    
    def latent(self, x):
        
        
        # 0. forward prop
        # ---------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)

        # 1. Working out layer number
        # ---------------------------
        if self.layer_mode == 'deep':
            forward_out = self.convl5(c4_out)
        
        elif self.layer_mode == 'deep_minus_2':
            forward_out = c3_out
        
        else:
            forward_out = c4_out


        # 2. Including a pool layer 
        # -------------------------
        ih,iw = forward_out.size()[2],forward_out.size()[3]
        
        if self.pool_mode == 'avg':

            # avg pool - comment/uncomment
            # ----------------------------
            avpl =  nn.Sequential(*[nn.AvgPool2d((ih,iw), stride=1)])
            latent_out = avpl(forward_out)
        
        elif self.pool_mode == 'max':
            
            # maxpool - comment/uncomment
            # ---------------------------
            mxpl =  nn.Sequential(*[nn.MaxPool2d((ih,iw), stride=1)])
            latent_out = mxpl(forward_out)
            
        elif self.pool_mode == 'both':
            
            # avg pool
            # --------
            avpl =  nn.Sequential(*[nn.AvgPool2d((ih,iw), stride=1)])
            latent_out_avg = avpl(forward_out)
            latent_out_avg = latent_out_avg.view(latent_out_avg.size()[0],-1)
            
            # maxpool
            # -------
            mxpl =  nn.Sequential(*[nn.MaxPool2d((ih,iw), stride=1)])
            latent_out_max = mxpl(forward_out)
            latent_out_max = latent_out_max.view(latent_out_max.size()[0],-1)
            
            # final concat
            # ------------
            latent_out = torch.cat((latent_out_avg, latent_out_max), 1)
            
        
        else:
            
            # no pooling
            # ----------
            latent_out = forward_out
            
        
        return latent_out.view(latent_out.size()[0],-1)
        
    
     
    def set_pool_mode(self, pool_mode, layer_mode):
        
        # setting pool mode
        # -----------------
        self.pool_mode = pool_mode
        self.layer_mode = layer_mode
        print('Modes set.')


In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_ae_6_layer_WNET(nn.Module):
    def __init__(self, in_channels, latent_softmax):
        super().__init__()
        
        # This is WNET model
        # ------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (191,191) -- Insize
        
        # @conv1 - (95,95)
        # @conv2 - (47, 47)
        # @conv3 - (23, 23)
        # @conv4 - (11, 11)
        # @conv5 - (5, 5)
        # @conv6 - (2,2)
        # Followed by a an avg pool (2,2) to make this 1,1
        
        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        dropout_prob = 0.2
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 16
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        conv4 = 128
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) 
        
        # Conv 5
        ###
        conv5 = 256
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) 
        
        # Conv 6
        ###
        conv6 = 512
        ct6 = nn.Conv2d(conv5,conv6,f,stride = s)
        cb6 = nn.BatchNorm2d(conv6)
        ca6 = nw_activation_conv
        cl6 = [ct6,cb6,ca6,dropout_node]
        self.convl6 = nn.Sequential(*cl6) 
        

        # Pooling layer + softmax activation
        # ----------------------------------
        if latent_softmax == True:
            avpl =  [nn.AvgPool2d((2,2), stride=1), nn.Softmax2d()]
        else:
            avpl =  [nn.AvgPool2d((2,2), stride=1)]
        self.pool_net = nn.Sequential(*avpl)
        
      
        # Transconv layers
        # ----------------
        # Showing conv up sizes - 
        # --------------------------
        # Incoming input is 1 x 1 x C
        # (5, 5)
        # (11, 11)
        # (23, 23)
        # (47, 47)
        # (95, 95)
        # (191, 191)
        
        # Upconv layer 0
        ###
        up_conv0 = conv6
        t0 = nn.ConvTranspose2d(conv6,up_conv0,2,stride = 1)
        b0 = nn.BatchNorm2d(up_conv0)
        a0 = nw_activation_conv
        l0 = [t0,b0,a0,dropout_node]
        self.upcl0 = nn.Sequential(*l0) # 2x2
        
        # Upconv layer 1
        # concat layer
        ###
        up_conv1 = 256
        t1 = nn.ConvTranspose2d(up_conv0 + conv6,up_conv1,f,stride = s)
        b1 = nn.BatchNorm2d(up_conv1)
        a1 = nw_activation_conv
        l1 = [t1,b1,a1,dropout_node]
        self.upcl1 = nn.Sequential(*l1) # 5x5
        
        # Upconv layer 2
        # concat layer
        ###
        up_conv2 = 128
        t2 = nn.ConvTranspose2d(up_conv1 + conv5,up_conv2,f,stride = s)
        b2 = nn.BatchNorm2d(up_conv2)
        a2 = nw_activation_conv
        l2 = [t2,b2,a2,dropout_node]
        self.upcl2 = nn.Sequential(*l2)
        
        # Upconv layer 3
        # concat layer
        ###
        up_conv3 = 64
        t3 = nn.ConvTranspose2d(up_conv2 + conv4,up_conv3,f,stride = s)
        b3 = nn.BatchNorm2d(up_conv3)
        a3 = nw_activation_conv
        l3 = [t3,b3,a3,dropout_node]
        self.upcl3 = nn.Sequential(*l3)
        
        # Upconv layer 4
        # concat layer
        ###
        up_conv4 = 32
        t4 = nn.ConvTranspose2d(up_conv3 + conv3,up_conv4,f,stride = s)
        b4 = nn.BatchNorm2d(up_conv4)
        a4 = nw_activation_conv
        l4 = [t4,b4,a4,dropout_node]
        self.upcl4 = nn.Sequential(*l4)
        
        # Upconv layer 5
        # concat layer
        ###
        up_conv5 = 16
        t5 = nn.ConvTranspose2d(up_conv4 + conv2,up_conv5,f,stride = s)
        b5 = nn.BatchNorm2d(up_conv5)
        a5 = nw_activation_conv
        l5 = [t5,b5,a5,dropout_node]
        self.upcl5 = nn.Sequential(*l5)
    
    
        # Upconv layer 6
        # concat layer - FINAL LAYER
        ###
        t6 = nn.ConvTranspose2d(up_conv5 + conv1,3,f,stride = s)
        a6 = nn.Sigmoid()
        l6 = [t6,a6]
        self.upcl6 = nn.Sequential(*l6)
        

    def forward(self, x):
        
        # Conv pass
        # ---------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        c6_out = self.convl6(c5_out)
        
        # pooling
        # -------
        latent_out = self.pool_net(c6_out)
        
        # Transconv pass
        # --------------
        f1_out = self.upcl0(latent_out)
        f2_out = self.upcl1(torch.cat((f1_out,c6_out), 1))
        f3_out = self.upcl2(torch.cat((f2_out,c5_out), 1))
        f4_out = self.upcl3(torch.cat((f3_out,c4_out), 1))
        f5_out = self.upcl4(torch.cat((f4_out,c3_out), 1))
        f6_out = self.upcl5(torch.cat((f5_out,c2_out), 1))
        f7_out = self.upcl6(torch.cat((f6_out,c1_out), 1))
        
        return f7_out

    
    def latent(self, x):
        
        
        # Conv pass
        # ---------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        c6_out = self.convl6(c5_out)
        
        # pooling
        # -------
        latent_out = self.pool_net(c6_out)

        return latent_out.view(latent_out.size()[0],-1)
        
    


In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class standard_cnn_6_layer(nn.Module):
    def __init__(self, in_channels, latent_softmax):
        super().__init__()
        
        # This is WNET model
        # ------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (191,191) -- Insize
        
        # @conv1 - (95,95)
        # @conv2 - (47, 47)
        # @conv3 - (23, 23)
        # @conv4 - (11, 11)
        # @conv5 - (5, 5)
        # @conv6 - (2,2)
        # Followed by a an avg pool (2,2) to make this 1,1
        
        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        dropout_prob = 0.2
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 16
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        conv4 = 128
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) 
        
        # Conv 5
        ###
        conv5 = 256
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) 
        
        # Conv 6
        ###
        conv6 = 512
        ct6 = nn.Conv2d(conv5,conv6,f,stride = s)
        cb6 = nn.BatchNorm2d(conv6)
        ca6 = nw_activation_conv
        cl6 = [ct6,cb6,ca6,dropout_node]
        self.convl6 = nn.Sequential(*cl6) 
        

        # Pooling layer + softmax activation
        # ----------------------------------
        if latent_softmax == True:
            avpl =  [nn.AvgPool2d((2,2), stride=1), nn.Softmax2d()]
        else:
            avpl =  [nn.AvgPool2d((2,2), stride=1)]
        self.pool_net = nn.Sequential(*avpl)
        
        
        # Adding linear layers
        # -------------------
        lnt1 = nn.Linear(conv6,256)
        lnb1 = nn.BatchNorm1d(256)
        lna1 = nw_activation_conv
        ln1 = [lnt1,lnb1,lna1,dropout_node]
        self.linear1 = nn.Sequential(*ln1) 
      
        lnt2 = nn.Linear(256,128)
        lnb2 = nn.BatchNorm1d(128)
        lna2 = nw_activation_conv
        ln2 = [lnt2,lnb2,lna2,dropout_node]
        self.linear2 = nn.Sequential(*ln2)
        
        lnt3 = nn.Linear(128,64)
        lnb3 = nn.BatchNorm1d(64)
        lna3 = nw_activation_conv
        ln3 = [lnt3,lnb3,lna3,dropout_node]
        self.linear3 = nn.Sequential(*ln3)
        
        ln4 = [nn.Linear(64,1)]
        self.linear4 = nn.Sequential(*ln4)
        

    def forward(self, x):
        
        # Conv pass
        # ---------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        c6_out = self.convl6(c5_out)
        
        # pooling
        # -------
        latent_out = self.pool_net(c6_out)
        
        # linear out
        # ----------
        linear1_out = self.linear1(latent_out.view(latent_out.size()[0],-1))
        linear2_out = self.linear2(linear1_out)
        linear3_out = self.linear3(linear2_out)
        linear4_out = self.linear4(linear3_out)
        
        
        
        return linear4_out


        
    


In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class standard_cnn_4_layer(nn.Module):
    def __init__(self, in_channels, latent_softmax):
        super().__init__()
        
        # This is WNET model
        # ------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (191,191) -- Insize
        
        # @conv1 - (95,95)
        # @conv2 - (47, 47)
        # @conv3 - (23, 23)
        # @conv4 - (11, 11)
        # @conv5 - (5, 5)
        # @conv6 - (2,2)
        # Followed by a an avg pool (2,2) to make this 1,1
        
        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2) # nn.Tanh() nn.Softmax2d()
        f = 3
        s = 2
        dropout_prob = 0.2
        dropout_node = nn.Dropout2d(p=dropout_prob)
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 64
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 128
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 256
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        conv4 = 512
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) 
    
        
        # Adding linear layers
        # -------------------
        lnt1 = nn.Linear(conv4*11*11,1024)
        lnb1 = nn.BatchNorm1d(1024)
        lna1 = nw_activation_conv
        ln1 = [lnt1,lnb1,lna1,dropout_node]
        self.linear1 = nn.Sequential(*ln1) 
      
        lnt2 = nn.Linear(1024,512)
        lnb2 = nn.BatchNorm1d(512)
        lna2 = nw_activation_conv
        ln2 = [lnt2,lnb2,lna2,dropout_node]
        self.linear2 = nn.Sequential(*ln2)
        
        lnt3 = nn.Linear(512,256)
        lnb3 = nn.BatchNorm1d(256)
        lna3 = nw_activation_conv
        ln3 = [lnt3,lnb3,lna3,dropout_node]
        self.linear3 = nn.Sequential(*ln3)
        
        ln4 = [nn.Linear(256,1)]
        self.linear4 = nn.Sequential(*ln4)
        

    def forward(self, x):
        
        # Conv pass
        # ---------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        
        # linear out
        # ----------
        linear1_out = self.linear1(c4_out.view(c4_out.size()[0],-1))
        linear2_out = self.linear2(linear1_out)
        linear3_out = self.linear3(linear2_out)
        linear4_out = self.linear4(linear3_out)
        
        return linear4_out


        
    


In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_ae_6_layer_UNET_multiple_outchannels(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        # This is an UNET model
        # ---------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (32,32) -- Insize
        
        # @conv1    -   (30,30)
        # @conv2    -   (28,28)
        # @maxpool1 -   (14,14)
        
        # @conv3    -   (12,12)
        # @conv4    -   (10,10)
        # @maxpool2 -   (5,5)
        
        # @conv5    -   (3,3)
        # @conv6    -   (1,1)

        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 3
        s = 1
        dropout_prob = 0.2
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 16
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1) # 30x30
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2) # 28x28
        
        # Maxpool 1
        ###
        mxpool_1 =  [nn.MaxPool2d(pool_dims, stride=1, return_indices=True)]
        self.mxpool1 = nn.Sequential(*mxpool_1) # 14x14
        

        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3) # 12x12
        
        # Conv 4
        ###
        conv4 = 128
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) # 10x10
        
        # Maxpool 2
        ###
        mxpool_2 =  [nn.MaxPool2d(pool_dims, stride=1, return_indices=True)]
        self.mxpool2 = nn.Sequential(*mxpool_2) # 5x5

        
        # Conv 5
        ###
        conv5 = 256
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) # 3x3
        
        # Conv 6
        ###
        conv6 = 512
        ct6 = nn.Conv2d(conv5,conv6,f,stride = s)
        cb6 = nn.BatchNorm2d(conv6)
        ca6 = nw_activation_conv
        cl6 = [ct6,cb6,ca6,dropout_node]
        self.convl6 = nn.Sequential(*cl6) # 1x1
        

      
        # Transconv layers
        # ----------------
        
        # Upconv layer 0
        ###
        up_conv0 = 256
        t0 = nn.ConvTranspose2d(conv6,up_conv0,f,stride = s)
        b0 = nn.BatchNorm2d(up_conv0)
        a0 = nw_activation_conv
        l0 = [t0,b0,a0,dropout_node]
        self.upcl0 = nn.Sequential(*l0) # 3x3
        
        # Upconv layer 1
        # concat layer
        ###
        up_conv1 = 128
        t1 = nn.ConvTranspose2d(up_conv0 + conv5,up_conv1,f,stride = s)
        b1 = nn.BatchNorm2d(up_conv1)
        a1 = nw_activation_conv
        l1 = [t1,b1,a1,dropout_node]
        self.upcl1 = nn.Sequential(*l1) # 5x5
        
        # Maxunpool 2
        ###
        mxunpool_2 =  [nn.MaxUnpool2d(pool_dims, stride=1)]
        self.mxunpool2 = nn.Sequential(*mxunpool_2) # 10x10
        
        
        
        # Upconv layer 2
        # concat layer
        ###
        up_conv2 = 64
        t2 = nn.ConvTranspose2d(up_conv1 + conv4,up_conv2,f,stride = s)
        b2 = nn.BatchNorm2d(up_conv2)
        a2 = nw_activation_conv
        l2 = [t2,b2,a2,dropout_node]
        self.upcl2 = nn.Sequential(*l2) # 12x12
        
        # Upconv layer 3
        # concat layer
        ###
        up_conv3 = 32
        t3 = nn.ConvTranspose2d(up_conv2 + conv3,up_conv3,f,stride = s)
        b3 = nn.BatchNorm2d(up_conv3)
        a3 = nw_activation_conv
        l3 = [t3,b3,a3,dropout_node]
        self.upcl3 = nn.Sequential(*l3) # 14x14
        
        # Maxunpool 1
        ###
        mxunpool_1 =  [nn.MaxUnpool2d(pool_dims, stride=1)]
        self.mxunpool1 = nn.Sequential(*mxunpool_1) # 28x28
        
        
        
        # Upconv layer 4
        # concat layer
        ###
        up_conv4 = 16
        t4 = nn.ConvTranspose2d(up_conv3 + conv2,up_conv4,f,stride = s)
        b4 = nn.BatchNorm2d(up_conv4)
        a4 = nw_activation_conv
        l4 = [t4,b4,a4,dropout_node]
        self.upcl4 = nn.Sequential(*l4) # 30x30
        
        
        # Upconv layer 5
        # concat layer - FINAL LAYER
        ###
        up_conv5 = out_channels
        t5 = nn.ConvTranspose2d(up_conv4 + conv1,up_conv5,f,stride = s)
        a5 = nn.Sigmoid()
        l5 = [t5,b5]
        self.upcl5 = nn.Sequential(*l5) # 32x32
    
        

    def forward(self, x):
        
        # Do over..
        
        
        ''' WONT WORK SINCE MAXUNPOOL CANNOT UNPOOL OVER NEWLY CONCATENATED CHANNELS 
        -- JUST DO A STD ONE WITH CON LAYERS ONLY '''
        
        
        
        return None
        
    


In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_classifier(nn.Module):
    def __init__(self, in_channels, out_channels, target_act):
        super().__init__()
        
        # This is an UNET model
        # ---------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (32,32) -- Insize
        
        # @conv1    -   (16,16)
        # @conv2    -   (8,8)        
        # @conv3    -   (4,4)
        # @conv4    -   (2,2)        
        # @conv5    -   (1,1)

        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 2
        s = 2
        dropout_prob = 0.05
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 64
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 128
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        conv4 = 256
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4)
        
        
        # Conv 5
        ###
        conv5 = out_channels
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        if target_act == 'sigmoid':
            ca5 = nn.Sigmoid()
        else:
            ca5 = nn.Softmax2d()
        cl5 = [ct5,ca5]
        self.convl5 = nn.Sequential(*cl5)

     
        

    def forward(self, x):
        
        # simple forward pass
        # -------------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        
        return c5_out




In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_do_not_use(nn.Module):
    def __init__(self):
        super().__init__()
        
        # This is ONLY FOR TEST PURPOSE
        # -----------------------------
        
        
        # Initialising N/W here
        # ---------------------
        in_channels, out_channels, target_act = 3,10,'sigmoid'
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 3
        s = 2
        dropout_prob = 0.2
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 16
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1)
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2)
        
        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3)
        
        # Conv 4
        ###
        conv4 = 128
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4)
        
        
        # Conv 5
        ###
        conv5 = out_channels
        ct5 = nn.Conv2d(conv4,conv5,f,stride = 1)
        if target_act == 'sigmoid':
            ca5 = nn.Sigmoid()
        else:
            ca5 = nn.Softmax2d()
        cl5 = [ct5,ca5]
        self.convl5 = nn.Sequential(*cl5)

     
        

    def forward(self, x):
        
        # simple forward pass
        # -------------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out)
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        
        return c5_out




In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_classifier_cifar10linear(nn.Module):
    def __init__(self, in_channels, out_channels, target_act):
        super().__init__()
        
        # This is an UNET model
        # ---------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (32,32) -- Insize
        
        # @conv1    -   (30,30)
        # @conv2    -   (28,28)
        # @maxpool1 -   (14,14)
        
        # @conv3    -   (12,12)
        # @conv4    -   (10,10)
        # @maxpool2 -   (5,5)
        
        # @conv5    -   (3,3)
        # @conv6    -   (1,1)

        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 3
        s = 1
        dropout_prob = 0.05
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1) # 30x30
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2) # 28x28
        
        # Maxpool 1
        ###
        mxpool_1 =  [nn.AvgPool2d(pool_dims, stride=2)]
        self.mxpool1 = nn.Sequential(*mxpool_1) # 14x14

        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3) # 12x12
        
        # Conv 4
        ###
        conv4 = 64
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) # 10x10
        
        # Maxpool 2
        ###
        mxpool_2 =  [nn.AvgPool2d(pool_dims, stride=2)]
        self.mxpool2 = nn.Sequential(*mxpool_2) # 5x5
        
        # Conv 5
        ###
        conv5 = 64
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) # 3X3
        
        # Conv 6
        ###
        conv6 = 64
        ct6 = nn.Conv2d(conv5,conv6,f,stride = s)
        cb6 = nn.BatchNorm2d(conv6)
        ca6 = nw_activation_conv
        cl6 = [ct6,cb6,ca6,dropout_node]
        self.convl6 = nn.Sequential(*cl6) # 1X1
        
        
        
        # linear layer 1
        ###
        ld1 = nn.Linear(1*1*64,32)
        lb1 = nn.BatchNorm1d(32)
        la1 = nw_activation_conv
        ln1 = [ld1,lb1,la1,dropout_node]
        self.lin1 = nn.Sequential(*ln1)
        
        # linear layer 2
        ###
        ld2 = nn.Linear(32,10)
        if target_act == 'sigmoid':
            la2 = nn.Sigmoid()
        else:
            la2 = nn.Softmax2d()
        ln2 = [ld2,la2]
        self.lin2 = nn.Sequential(*ln2)
        
        
        
        
    def forward(self, x):
        
        # simple forward pass
        # -------------------
        c1_out = self.convl1(x) # 30X30
        c2_out = self.convl2(c1_out) # 28x28
        mxpool1_out = self.mxpool1(c2_out) # 14x14
        
        
        c3_out = self.convl3(mxpool1_out) # 12x12
        c4_out = self.convl4(c3_out) #10x10
        mxpool2_out = self.mxpool2(c4_out) #5x5
        
        c5_out = self.convl5(mxpool2_out) #3x3
        c6_out = self.convl6(c5_out) #1x1
        
        # linear pass
        # -----------
        l1_out = self.lin1(c6_out.view(c6_out.size()[0],-1))
        l2_out = self.lin2(l1_out)
        
        return l2_out

In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_classifier_cifar10(nn.Module):
    def __init__(self, in_channels, out_channels, target_act):
        super().__init__()
        
        # This is an UNET model
        # ---------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (32,32) -- Insize
        
        # @conv1    -   (30,30)
        # @conv2    -   (28,28)
        # @maxpool1 -   (14,14)
        
        # @conv3    -   (12,12)
        # @conv4    -   (10,10)
        # @maxpool2 -   (5,5)
        
        # @conv5    -   (3,3)
        # @conv6    -   (1,1)

        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.ReLU() #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 3
        s = 1
        dropout_prob = 0.1
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1) # 30x30
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2) # 28x28
        
        # Maxpool 1
        ###
        mxpool_1 =  [nn.MaxPool2d(pool_dims, stride=2)]
        self.mxpool1 = nn.Sequential(*mxpool_1) # 14x14

        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3) # 12x12
        
        # Conv 4
        ###
        conv4 = 64
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) # 10x10
        
        # Maxpool 2
        ###
        mxpool_2 =  [nn.MaxPool2d(pool_dims, stride=2)]
        self.mxpool2 = nn.Sequential(*mxpool_2) # 5x5
        
        # Conv 5
        ###
        conv5 = 128
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) # 3x3
        
        
        # Conv 6
        ###
        ct6 = nn.Conv2d(conv5,out_channels,f,stride = s)
        if target_act == 'sigmoid':
            ca6 = nn.Sigmoid()
        else:
            ca6 = nn.Softmax2d()
        cl6 = [ct6,ca6]
        self.convl6 = nn.Sequential(*cl6) # 1x1
        
        
        
    def forward(self, x):
        
        # simple forward pass
        # -------------------
        c1_out = self.convl1(x) # 30X30
        c2_out = self.convl2(c1_out) # 28x28
        mxpool1_out = self.mxpool1(c2_out) # 14x14
        
        
        c3_out = self.convl3(mxpool1_out) # 12x12
        c4_out = self.convl4(c3_out) #10x10
        mxpool2_out = self.mxpool2(c4_out) #5x5
        
        
        c5_out = self.convl5(mxpool2_out) # 3x3
        c6_out = self.convl6(c5_out) # 1x1
        
        return c6_out

In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_classifier_cifar10_linlayers(nn.Module):
    def __init__(self, in_channels, out_channels, target_act):
        super().__init__()
        
        # This is an UNET model
        # ---------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (32,32) -- Insize
        
        # @conv1    -   (30,30)
        # @conv2    -   (28,28)
        # @maxpool1 -   (14,14)
        
        # @conv3    -   (12,12)
        # @conv4    -   (10,10)
        # @maxpool2 -   (5,5)
        
        # @conv5    -   (3,3)
        # @conv6    -   (1,1)

        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.LeakyReLU(0.2) #nn.ReLU() #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 3
        s = 1
        dropout_prob = 0.05
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1) # 30x30
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2) # 28x28
        
        # Maxpool 1
        ###
        mxpool_1 =  [nn.MaxPool2d(pool_dims, stride=2)]
        self.mxpool1 = nn.Sequential(*mxpool_1) # 14x14

        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3) # 12x12
        
        # Conv 4
        ###
        conv4 = 64
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) # 10x10
        
        # Maxpool 2
        ###
        mxpool_2 =  [nn.MaxPool2d(pool_dims, stride=2)]
        self.mxpool2 = nn.Sequential(*mxpool_2) # 5x5
        
        # Conv 5
        ###
        conv5 = 128
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) # 3x3
        
        
        # Conv 6
        ###
        conv6 = 128
        ct6 = nn.Conv2d(conv5,conv6,f,stride = s)
        cb6 = nn.BatchNorm2d(conv6)
        ca6 = nw_activation_conv
        cl6 = [ct6,cb6,ca6,dropout_node]
        self.convl6 = nn.Sequential(*cl6) # 1x1
        
        
        
        #################### Linear 1x1 convs #########################
        
        # Conv ln 1
        ###
        convln1 = 64
        ctln1 = nn.Conv2d(conv6,convln1,1,stride = 1)
        cbln1 = nn.BatchNorm2d(convln1)
        caln1 = nw_activation_conv
        clln1 = [ctln1,cbln1,caln1,dropout_node]
        self.convlln1 = nn.Sequential(*clln1) # 1x1
        
        # Conv ln 2
        ###
        convln2 = 32
        ctln2 = nn.Conv2d(convln1,convln2,1,stride = 1)
        cbln2 = nn.BatchNorm2d(convln2)
        caln2 = nw_activation_conv
        clln2 = [ctln2,cbln2,caln2,dropout_node]
        self.convlln2 = nn.Sequential(*clln2) # 1x1
        
        # Conv ln 3
        ###
        #convln3 = 64
        #ctln3 = nn.Conv2d(convln2,convln3,1,stride = 1)
        #cbln3 = nn.BatchNorm2d(convln3)
        #caln3 = nw_activation_conv
        #clln3 = [ctln3,cbln3,caln3,dropout_node]
        #self.convlln3 = nn.Sequential(*clln3) # 1x1
        
        # Conv ln 4
        ###
        #convln4 = 32
        #ctln4 = nn.Conv2d(convln3,convln4,1,stride = 1)
        #cbln4 = nn.BatchNorm2d(convln4)
        #caln4 = nw_activation_conv
        #clln4 = [ctln4,cbln4,caln4,dropout_node]
        #self.convlln4 = nn.Sequential(*clln4) # 1x1
        
        
        
        #################### final output conv #########################
        
        
        # Conv 6
        ###
        ct7 = nn.Conv2d(convln2,out_channels,1,stride = 1)
        if target_act == 'sigmoid':
            ca7 = nn.Sigmoid()
        else:
            ca7 = nn.Softmax2d()
        cl7 = [ct7,ca7]
        self.convl7 = nn.Sequential(*cl7) # 1x1
        
        
        
    def forward(self, x):
        
        # simple forward pass
        # -------------------
        c1_out = self.convl1(x) # 30X30
        c2_out = self.convl2(c1_out) # 28x28
        mxpool1_out = self.mxpool1(c2_out) # 14x14
        
        c3_out = self.convl3(mxpool1_out) # 12x12
        c4_out = self.convl4(c3_out) #10x10
        mxpool2_out = self.mxpool2(c4_out) #5x5
        
        c5_out = self.convl5(mxpool2_out) # 3x3
        c6_out = self.convl6(c5_out) # 1x1
        
        # linear layer passes
        # -------------------
        cln1_out = self.convlln1(c6_out) # 1x1
        cln2_out = self.convlln2(cln1_out) # 1x1
        #cln3_out = self.convlln3(cln2_out) # 1x1
        #cln4_out = self.convlln4(cln3_out) # 1x1
        
        # final out
        # ---------
        c7_out = self.convl7(cln2_out) # 1x1
        
        
        return c7_out

In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_classifier_cifar10_deeper(nn.Module):
    def __init__(self, in_channels, out_channels, target_act):
        super().__init__()
        
        # This is an UNET model
        # ---------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (32,32) -- Insize
        
        # @conv1    -   (30,30)
        # @conv2    -   (28,28)
        # @conv3    -   (26,26)
        # @conv4    -   (24,24)
        # @conv5    -   (22,22)
        # @maxpool1 -   (11,11)
        

        # @conv6    -   (9,9)
        # @conv7    -   (7,7)
        # @conv8    -   (5,5)
        # @conv9    -   (3,3)
        # @conv10    -   (1,1)
        # @conv11    -   (1,1)
        
        
        
        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.LeakyReLU(0.2) #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 3
        s = 1
        dropout_prob = 0.0
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1) 
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2) 

        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3) 
        
        # Conv 4
        ###
        conv4 = 64
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) 
        
        # Conv 5
        ###
        conv5 = 128
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) 
        
        
        # Maxpool 1
        ###
        mxpool_1 =  [nn.MaxPool2d(pool_dims, stride=2)]
        self.mxpool1 = nn.Sequential(*mxpool_1) # 14x14
        
        
        # Conv 6
        ###
        conv6 = 128
        ct6 = nn.Conv2d(conv5,conv6,f,stride = s)
        cb6 = nn.BatchNorm2d(conv6)
        ca6 = nw_activation_conv
        cl6 = [ct6,cb6,ca6,dropout_node]
        self.convl6 = nn.Sequential(*cl6) 
        
        
        # Conv 7
        ###
        conv7 = 256
        ct7 = nn.Conv2d(conv6,conv7,f,stride = s)
        cb7 = nn.BatchNorm2d(conv7)
        ca7 = nw_activation_conv
        cl7 = [ct7,cb7,ca7,dropout_node]
        self.convl7 = nn.Sequential(*cl7) 
        
        # Conv 8
        ###
        conv8 = 256
        ct8 = nn.Conv2d(conv7,conv8,f,stride = s)
        cb8 = nn.BatchNorm2d(conv8)
        ca8 = nw_activation_conv
        cl8 = [ct8,cb8,ca8,dropout_node]
        self.convl8 = nn.Sequential(*cl8) 
        
        # Conv 9
        ###
        conv9 = 512
        ct9 = nn.Conv2d(conv8,conv9,f,stride = s)
        cb9 = nn.BatchNorm2d(conv9)
        ca9 = nw_activation_conv
        cl9 = [ct9,cb9,ca9,dropout_node]
        self.convl9 = nn.Sequential(*cl9) 
        
        # Conv 10
        ###
        conv10 = 512
        ct10 = nn.Conv2d(conv9,conv10,f,stride = s)
        cb10 = nn.BatchNorm2d(conv10)
        ca10 = nw_activation_conv
        cl10 = [ct10,cb10,ca10,dropout_node]
        self.convl10 = nn.Sequential(*cl10) 
        
        
        
        
        
        #################### Linear 1x1 convs #########################
        
        # Conv ln 1
        ###
        convln1 = 1024
        ctln1 = nn.Conv2d(conv10,convln1,1,stride = 1)
        cbln1 = nn.BatchNorm2d(convln1)
        caln1 = nw_activation_conv
        clln1 = [ctln1,cbln1,caln1,dropout_node]
        self.convlln1 = nn.Sequential(*clln1) # 1x1
        
        # Conv ln 2
        ###
        convln2 = 512
        ctln2 = nn.Conv2d(convln1,convln2,1,stride = 1)
        cbln2 = nn.BatchNorm2d(convln2)
        caln2 = nw_activation_conv
        clln2 = [ctln2,cbln2,caln2,dropout_node]
        self.convlln2 = nn.Sequential(*clln2) # 1x1
        
        # Conv ln 3
        ###
        convln3 = 256
        ctln3 = nn.Conv2d(convln2,convln3,1,stride = 1)
        cbln3 = nn.BatchNorm2d(convln3)
        caln3 = nw_activation_conv
        clln3 = [ctln3,cbln3,caln3,dropout_node]
        self.convlln3 = nn.Sequential(*clln3) # 1x1
        
        # Conv ln 4
        ###
        convln4 = 128
        ctln4 = nn.Conv2d(convln3,convln4,1,stride = 1)
        cbln4 = nn.BatchNorm2d(convln4)
        caln4 = nw_activation_conv
        clln4 = [ctln4,cbln4,caln4,dropout_node]
        self.convlln4 = nn.Sequential(*clln4) # 1x1
        
        
        ##################### final out ########################
        
        # Conv 11
        ###
        ct11 = nn.Conv2d(convln4,out_channels,1,stride = 1)
        if target_act == 'sigmoid':
            ca11 = nn.Sigmoid()
        else:
            ca11 = nn.Softmax2d()
        cl11 = [ct11,ca11]
        self.convl11 = nn.Sequential(*cl11) # 1x1
        
        
        
    def forward(self, x):
        
        # simple forward pass
        # -------------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out) 
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        
        mxpool1_out = self.mxpool1(c5_out)
        
        c6_out = self.convl6(mxpool1_out)
        c7_out = self.convl7(c6_out) 
        c8_out = self.convl8(c7_out)
        c9_out = self.convl9(c8_out)
        c10_out = self.convl10(c9_out)
        
        # linear layer passes
        # -------------------
        cln1_out = self.convlln1(c10_out) # 1x1
        cln2_out = self.convlln2(cln1_out) # 1x1
        cln3_out = self.convlln3(cln2_out) # 1x1
        cln4_out = self.convlln4(cln3_out) # 1x1
        
        
        c11_out = self.convl11(cln4_out)
        
        return c11_out
    
    

In [ ]:
# FCN class copied from image search notebook which worked
# --------------------------------------------------------

class fcn_classifier_cifar10_exo_deep(nn.Module):
    def __init__(self, in_channels, out_channels, target_act):
        super().__init__()
        
        # This is an UNET model
        # ---------------------
        
        # Showing conv up sizes - 
        # --------------------------
        # (32,32) -- Insize
        
        # @conv1    -   (30,30)
        # @conv2    -   (28,28)
        # @conv3    -   (26,26)
        # @conv4    -   (24,24)
        # @conv5    -   (22,22)
        
        # avgppool - (1,1)
        
        # @conv6    -   (9,9)
        # @conv7    -   (7,7)
        # @conv8    -   (5,5)
        # @conv9    -   (3,3)
        # @conv10    -   (1,1)
        # @conv11    -   (1,1)
        
        
        
        
        
        # Initialising N/W here
        # ---------------------
        nw_activation_conv = nn.LeakyReLU(0.2) #nn.LeakyReLU(0.2), nn.Tanh(), nn.Softmax2d()
        f = 3
        s = 1
        dropout_prob = 0.0
        dropout_node = nn.Dropout2d(p=dropout_prob)
        pool_dims = (2,2)
        
        
        
        # CONV Down layers
        # ----------------
        
        # Conv 1
        ###
        conv1 = 32
        ct1 = nn.Conv2d(in_channels,conv1,f,stride = s)
        cb1 = nn.BatchNorm2d(conv1)
        ca1 = nw_activation_conv
        cl1 = [ct1,cb1,ca1,dropout_node]
        self.convl1 = nn.Sequential(*cl1) 
        
        # Conv 2
        ###
        conv2 = 32
        ct2 = nn.Conv2d(conv1,conv2,f,stride = s)
        cb2 = nn.BatchNorm2d(conv2)
        ca2 = nw_activation_conv
        cl2 = [ct2,cb2,ca2,dropout_node]
        self.convl2 = nn.Sequential(*cl2) 

        # Conv 3
        ###
        conv3 = 64
        ct3 = nn.Conv2d(conv2,conv3,f,stride = s)
        cb3 = nn.BatchNorm2d(conv3)
        ca3 = nw_activation_conv
        cl3 = [ct3,cb3,ca3,dropout_node]
        self.convl3 = nn.Sequential(*cl3) 
        
        # Conv 4
        ###
        conv4 = 64
        ct4 = nn.Conv2d(conv3,conv4,f,stride = s)
        cb4 = nn.BatchNorm2d(conv4)
        ca4 = nw_activation_conv
        cl4 = [ct4,cb4,ca4,dropout_node]
        self.convl4 = nn.Sequential(*cl4) 
        
        # Conv 5
        ###
        conv5 = 64
        ct5 = nn.Conv2d(conv4,conv5,f,stride = s)
        cb5 = nn.BatchNorm2d(conv5)
        ca5 = nw_activation_conv
        cl5 = [ct5,cb5,ca5,dropout_node]
        self.convl5 = nn.Sequential(*cl5) 
        
        # Conv 6
        ###
        conv6 = 128
        ct6 = nn.Conv2d(conv5,conv6,f,stride = s)
        cb6 = nn.BatchNorm2d(conv6)
        ca6 = nw_activation_conv
        cl6 = [ct6,cb6,ca6,dropout_node]
        self.convl6 = nn.Sequential(*cl6) 
        
        
        # Conv 7
        ###
        conv7 = 128
        ct7 = nn.Conv2d(conv6,conv7,f,stride = s)
        cb7 = nn.BatchNorm2d(conv7)
        ca7 = nw_activation_conv
        cl7 = [ct7,cb7,ca7,dropout_node]
        self.convl7 = nn.Sequential(*cl7) 
        
        # Conv 8
        ###
        conv8 = 256
        ct8 = nn.Conv2d(conv7,conv8,f,stride = s)
        cb8 = nn.BatchNorm2d(conv8)
        ca8 = nw_activation_conv
        cl8 = [ct8,cb8,ca8,dropout_node]
        self.convl8 = nn.Sequential(*cl8) 
        
        # Conv 9
        ###
        conv9 = 256
        ct9 = nn.Conv2d(conv8,conv9,f,stride = s)
        cb9 = nn.BatchNorm2d(conv9)
        ca9 = nw_activation_conv
        cl9 = [ct9,cb9,ca9,dropout_node]
        self.convl9 = nn.Sequential(*cl9) 
        
        # Conv 10
        ###
        conv10 = 512
        ct10 = nn.Conv2d(conv9,conv10,14,stride = 1)
        cb10 = nn.BatchNorm2d(conv10)
        ca10 = nw_activation_conv
        cl10 = [ct10,cb10,ca10,dropout_node]
        self.convl10 = nn.Sequential(*cl10) 
        
        
        
        # Maxpool 1
        ###
        mxpool_1 =  [nn.AvgPool2d((12,12), stride=1)]
        self.mxpool1 = nn.Sequential(*mxpool_1) # 14x14
        
        
        
        
        #################### Linear 1x1 convs #########################
        
        # Conv ln 1
        ###
        convln1 = 256
        ctln1 = nn.Conv2d(conv10,convln1,1,stride = 1)
        cbln1 = nn.BatchNorm2d(convln1)
        caln1 = nw_activation_conv
        clln1 = [ctln1,cbln1,caln1,dropout_node]
        self.convlln1 = nn.Sequential(*clln1) # 1x1
        
        # Conv ln 2
        ###
        convln2 = 128
        ctln2 = nn.Conv2d(convln1,convln2,1,stride = 1)
        cbln2 = nn.BatchNorm2d(convln2)
        caln2 = nw_activation_conv
        clln2 = [ctln2,cbln2,caln2,dropout_node]
        self.convlln2 = nn.Sequential(*clln2) # 1x1
        
        # Conv ln 3
        ###
        convln3 = 64
        ctln3 = nn.Conv2d(convln2,convln3,1,stride = 1)
        cbln3 = nn.BatchNorm2d(convln3)
        caln3 = nw_activation_conv
        clln3 = [ctln3,cbln3,caln3,dropout_node]
        self.convlln3 = nn.Sequential(*clln3) # 1x1
        
        # Conv ln 4
        ###
        convln4 = 32
        ctln4 = nn.Conv2d(convln3,convln4,1,stride = 1)
        cbln4 = nn.BatchNorm2d(convln4)
        caln4 = nw_activation_conv
        clln4 = [ctln4,cbln4,caln4,dropout_node]
        self.convlln4 = nn.Sequential(*clln4) # 1x1
        
        
        ##################### final out ########################
        
        # Conv 11
        ###
        ct11 = nn.Conv2d(conv10,out_channels,1,stride = 1)
        if target_act == 'sigmoid':
            ca11 = nn.Sigmoid()
        else:
            ca11 = nn.Softmax2d()
        cl11 = [ct11,ca11]
        self.convl11 = nn.Sequential(*cl11) # 1x1
        
        
        
    def forward(self, x):
        
        # simple forward pass
        # -------------------
        c1_out = self.convl1(x)
        c2_out = self.convl2(c1_out)
        c3_out = self.convl3(c2_out) 
        c4_out = self.convl4(c3_out)
        c5_out = self.convl5(c4_out)
        c6_out = self.convl6(c5_out)
        c7_out = self.convl7(c6_out) 
        c8_out = self.convl8(c7_out)
        c9_out = self.convl9(c8_out)
        c10_out = self.convl10(c9_out)
        
        
        #mxpool1_out = self.mxpool1(c10_out)
        
        
        # linear layer passes
        # -------------------
        #cln1_out = self.convlln1(mxpool1_out) # 1x1
        #cln2_out = self.convlln2(cln1_out) # 1x1
        #cln3_out = self.convlln3(cln2_out) # 1x1
        #cln4_out = self.convlln4(cln3_out) # 1x1
        
        
        c11_out = self.convl11(c10_out)
        
        return c11_out
    
    

# See All codes

In [ ]:
# parent function
# ---------------


def see_all(mode,model_in,xin,target_act):
    
    '''
    
    Purpose of this function is to iterate over new kernel sizes and suggest dims changes
    
    '''
    
    # needed assertion
    # ----------------
    assert mode != 'all', 'This function works only for first layer weights resize, not all.'
    
    
    # 0. initialisations
    # ------------------
    _,in_h,in_w,_ = xin.shape
    model_seq = []
    d = {}
    d[0] = {}
    d[0]['f'] = []
    d[0]['s_list'] = []
    
    
    # 0.1 initial setups
    # ------------------
    model_1temp = copy.deepcopy(model_in)
    for each in model_1temp.children():
        model_seq += list(each)
    d[0]['model'] = nn.Sequential(*model_seq)
        
    # 0.2 capturing kernel size & strides
    # -----------------------------------
    for each in model_seq:
        if 'Conv2d' in str(type(each)):
            d[0]['f'].append(each.kernel_size[0])
            d[0]['s_list'].append(each.stride[0])
 
    # 1. running master size ops on initial inputs
    # --------------------------------------------
    print('At increment step 1...', end=' ')
    conv_h,conv_w = in_h,in_w
    for i in range(len(d[0]['s_list'])):
        conv_h,conv_w = outsize_conv(conv_h,conv_w,d[0]['f'][i],d[0]['s_list'][i],0)
        
    # CHECK DIMS
    # ----------
    if conv_h < 1 or conv_w < 1:
        
        assert 1 == 2, 'Input dimensions not large enough for original model. Resizing op not coded yet.'
    
    else:
        
        # rounding and preparing new dims
        # -------------------------------
        req_h = round(conv_h)
        req_w = round(conv_w)
        
        # getting new dims
        # ----------------
        for i in range(len(d[0]['s_list'])):
            
            temporary_s = list(reversed(d[0]['s_list']))[i]
            temporary_f = list(reversed(d[0]['f']))[i]
            req_h,req_w = outsize_upconv(req_h,req_w,temporary_f,temporary_s,0)
        
        # saving new dims
        # ---------------
        d[0]['new_dims'] = (req_h,req_w)
        print(' done.', end='\n')
        
    
    # 2. Iterating over new f's with incremental 1
    # --------------------------------------------
    recurr_f = d[0]['f'][0]
    counter = 1
    while True:
        
        # 2.0 sanity printing
        # -------------------
        print('At increment step ' + str(counter+1) + '...', end=' ')
        
        # 2.1 incrementing f
        # -------------------
        recurr_f += 1
        
        # 2.2 creating new model with new f
        # ---------------------------------
        temp_d = resize_model(mode,copy.deepcopy(model_in),recurr_f)
        
        # 2.3 running master size ops on recurring inputs
        # -----------------------------------------------
        conv_h,conv_w = in_h,in_w
        for s in temp_d['s_list']:
            temporary_f = s[0]
            temporary_s = s[1]
            conv_h,conv_w = outsize_conv(conv_h,conv_w,temporary_f,temporary_s,0)

        # CHECK DIMS
        # ----------
        if conv_h < 2 or conv_w < 2:

            print(' Stopped.', end='\n')
            break

        else:

            # rounding and preparing new dims
            # -------------------------------
            req_h = round(conv_h)
            req_w = round(conv_w)

            # getting new dims
            # ----------------
            for s in list(reversed(temp_d['s_list'])):
                temporary_f = s[0]
                temporary_s = s[1]
                req_h,req_w = outsize_upconv(req_h,req_w,temporary_f,temporary_s,0)

            # saving new dims
            # ---------------
            d[counter] = {}
            d[counter]['model'] = temp_d['new_model']
            d[counter]['new_dims'] = (req_h,req_w)
            d[counter]['f'] = []
            d[counter]['s_list'] = []
            
            # saving f,s
            # ----------
            for s in temp_d['s_list']:
                d[counter]['f'].append(s[0])
                d[counter]['s_list'].append(s[1])

            # increment counter
            # -----------------
            counter += 1
            print(' done.', end='\n')
        
        
        
    # 3. doing all forward passes with this section
    # ---------------------------------------------
    output_dict = {}
    print('\nForward passing all...\n--------------------')
    for keys in d:
        
        # sanity print
        # ------------
        print('At new model ' + str(keys+1) + ' of ' + str(len(d)) + '..')
        
        # initialising tempd
        # ------------------
        temp_d = {}
        temp_d[keys] = d[keys]
        try:
            recurr_sum += forward_pass_all(temp_d,xin,target_act)
            #recurr_sum = torch.max(recurr_sum,forward_pass_all(temp_d,xin,target_act))
            
        except:
            recurr_sum = forward_pass_all(temp_d,xin,target_act)
        
    print('done.')

    
    # a final normalisation to 0-1
    # ----------------------------
    for i in range(recurr_sum.shape[0]):
        recurr_sum[i] = recurr_sum[i]/np.max(recurr_sum[i])
    
    # makes a bit more sense to normalise this way
    ##
    #recurr_sum[recurr_sum < np.mean(recurr_sum)] = 0
    #recurr_sum = recurr_sum/np.max(recurr_sum)
    
    return recurr_sum, d




In [ ]:
# function to actually see all
# ----------------------------

def forward_pass_all(d,xin,target_act):
    
    '''
    
    1. takes as input dict with resized models and xin (m,h,w,c) numpy array
    2. forward passes all images in the array and returns a (m,h,w,no_classes) output
    
    '''
    
    # 0. initialisations
    # ------------------
    m,h,w,c = xin.shape
    
    # 0. iterating
    # ------------
    for i in range(m):
        
        # 0.1 local initialisations
        # -------------------------
        x_curr = copy.deepcopy(xin[i])
        
        
        # 1.1 iterating trough the model
        # ------------------------------
        for keys in d:
            
            # more local initialisations
            # --------------------------
            curr_model = d[keys]['model']
            curr_h,curr_w = d[keys]['new_dims']
            x_curr = cv2.resize(x_curr, (curr_w,curr_h))
            x_curr = x_curr.reshape(1,curr_h,curr_w,3)
            x_curr_trn = Variable(setup_image_tensor(x_curr)).float()
            x_curr_trn = x_curr_trn/255
            
            # forward pass
            # ------------
            curr_out = curr_model(x_curr_trn)[0]
            curr_out = curr_out.data.numpy()
            
            # axis swap prior to resize since resize required (h,w,c) shape
            # -------------------------------------------------------------
            curr_out = np.swapaxes(curr_out,0,2)
            curr_out = np.swapaxes(curr_out,0,1)
            no_channels = curr_out.shape[2]
            
            # we could include a target active code here if required
            ###
            curr_out[curr_out < 0.75] = 0.001
            
            #if target_act == 'sigmoid':
                #curr_out[curr_out < 0.5] = 0
                #curr_out[curr_out >= 0.5] = 1
                #pass
                
            #else:
                #curr_out[curr_out < 0.5] = 0
                #curr_out[curr_out >= 0.5] = 1
                #pass
                

            # resize
            # ------
            curr_out = cv2.resize(curr_out,(w,h))#, interpolation=cv2.INTER_NEAREST)
            curr_out = curr_out.reshape(1,h,w,no_channels)
            
            
            # adding the outputs of each see all model
            # ----------------------------------------
            try:
                local_key_out += curr_out
            except:
                local_key_out = curr_out
           
                
            
        
        # 1.2 outside see all model keys
        # ------------------------------
        try:
            final_out = np.concatenate((final_out,local_key_out), axis = 0)
        except:
            final_out = local_key_out
        
        
        # 1.3 clearning local_key_out to start fresh for other images
        # -----------------------------------------------------------
        del local_key_out
        #print('Done with images ' + str(i+1) + ' of ' + str(m) + '..', end='\r')

            
    # final return
    # ------------
    return final_out
    
    


In [ ]:
# Function that resizes trained model
# -----------------------------------



def resize_model(mode,model_in,new_f):
    
    '''
    
    1. easy formula for working out strides at each layer
    2. using cv2.resize with nearest neighbour method to resize weights
    3. assigns new weights and strides and returns a new model
    4. also returns new input image size to be used based on required output size
    
    
    A way to check weights 
    ----------------------
    index = 8
    out_channel = 24
    in_channel = 21

    oldmodel[index].weight[out_channel,in_channel,:,:]
    newmodel[index].weight[out_channel,in_channel,:,:]

    '''
    
    
    # 0. initialisations
    # ------------------
    model_seq = []
    old_model = []
    strides_list = []
    mode_counter = 0
    mode_flag = 0
    
    
    # 1. building a sequential model from input
    # -----------------------------------------
    model_1temp = copy.deepcopy(model_in)
    model_2temp = copy.deepcopy(model_in)

    for each in model_1temp.children():
        model_seq += list(each)
    
    for each in model_2temp.children():
        old_model += list(each)
    
    
    # 2. iterating through each model and working out new stride values
    # -----------------------------------------------------------------
    for each in model_seq:
        if 'Conv2d' in str(type(each)) or 'MaxPool2d' in str(type(each)) or 'AvgPool2d' in str(type(each)):
            
            # mode counter updation
            # ---------------------
            mode_counter += 1
            
            # setting mode to single automatically if max or avgpool found in NW 
            # ------------------------------------------------------------------
            if 'MaxPool2d' in str(type(each)) or 'AvgPool2d' in str(type(each)):
                mode = 'single'
            
            # checking mode
            # -------------
            if mode == 'all':
                mode_flag = 1
            else:
                if mode_counter == 1:
                    mode_flag = 1
                else:
                    mode_flag = 0
            
            if mode_flag == 1:
            
                # updating stride
                # ---------------
                curr_f = each.kernel_size[0]
                curr_s = each.stride[0]

                if mode == 'all':
                
                    # using math.floor here to floor down strides
                    # -------------------------------------------
                    new_s = math.floor((curr_s/curr_f)*new_f)
                    
                else:
                    # using round to up strides
                    # -------------------------------------------
                    new_s = round((curr_s/curr_f)*new_f)
                    #new_s = math.floor((curr_s/curr_f)*new_f)
                    
                

                # resizing and updating weights
                # -----------------------------
                curr_weight = each.weight
                for c in range(curr_weight.size()[0]):
                    curr_weight_channel_orig = curr_weight[c].data.numpy()
                    curr_weight_channel = copy.deepcopy(curr_weight_channel_orig)

                    # axis swap prior to resize since resize required (h,w,c) shape
                    # -------------------------------------------------------------
                    curr_weight_channel = np.swapaxes(curr_weight_channel,0,2)
                    curr_weight_channel = np.swapaxes(curr_weight_channel,0,1)

                    # resize
                    # ------
                    updated_weight_channel = cv2.resize(curr_weight_channel,(new_f,new_f), interpolation=cv2.INTER_NEAREST)

                    # axis swap after reshape - back to (c,h,w)
                    # -----------------------------------------
                    updated_weight_channel = np.swapaxes(updated_weight_channel,0,2)
                    updated_weight_channel = np.swapaxes(updated_weight_channel,1,2)
                    updated_weight_channel = updated_weight_channel.reshape(1,updated_weight_channel.shape[0],updated_weight_channel.shape[1],updated_weight_channel.shape[2])

                    # concatenating
                    # -------------
                    try:
                        new_weight = np.concatenate((new_weight,updated_weight_channel), axis = 0)
                    except:
                        new_weight = updated_weight_channel

                # converting numpy weight to torch and setting it back to NW
                # -----------------------------------------------------------
                each.weight = torch.nn.Parameter(torch.from_numpy(new_weight))
                each.kernel_size = (new_f,new_f)
                each.stride = new_s
                strides_list.append((new_f,new_s))
            
            else:
                
                # we are not updating kernel size or stride this is section
                # ---------------------------------------------------------
                curr_f = each.kernel_size[0]
                try:
                    curr_s = each.stride[0]
                except:
                    curr_s = each.stride
                    
                strides_list.append((curr_f,curr_s))
                



    # final return
    # ------------
    d = {}
    d['new_model'] = nn.Sequential(*model_seq)
    d['old_model'] = nn.Sequential(*old_model)
    d['s_list'] = strides_list
    
    
    return d
    

In [ ]:
# END OF CODE

# 1. loading dataset

### 1.1 setting up model related variables

In [ ]:
## SET UP CUDA OR NOT HERE + OTHER SET UPS
##########################################

dev_env = 'local' # 'gpu' or 'local'

##########################################
##########################################

# Setting CUDA
# ------------
if dev_env == 'gpu':
    use_cuda = True
else:
    use_cuda = False
if use_cuda == True:
    torch.cuda.empty_cache()
    

# SET FILE SPECIFIC NAMES HERE
# ----------------------------
if dev_env == 'gpu':
    save_path = '/home/venkateshmadhava/codes/pmate2_localgpuenv/models/'
    parent_url = '/home/venkateshmadhava/datasets/images'
else:
    save_path = '/Users/venkateshmadhava/Documents/pmate2/pmate2_env/models/'
    parent_url = '/Users/venkateshmadhava/Documents/projects/vision/object_detection/cifar_10_batches_py'


# displaying save path
# --------------------
print(save_path)

In [ ]:
# Loading classes
# ---------------

labels_head = unpickle(parent_url + '/' + 'batches.meta')
labels_head

In [ ]:
# Loading batches of data & concatenating them
# --------------------------------------------

# loading batchs
# --------------
btch1 = parent_url + '/data_batch_1'
load_cifar10(btch1)
x1 = copy.deepcopy(x)
y1 = copy.deepcopy(y)

##
btch1 = parent_url + '/data_batch_2'
load_cifar10(btch1)
x2 = copy.deepcopy(x)
y2 = copy.deepcopy(y)

##
btch1 = parent_url + '/data_batch_3'
load_cifar10(btch1)
x3 = copy.deepcopy(x)
y3 = copy.deepcopy(y)

##
btch1 = parent_url + '/data_batch_4'
load_cifar10(btch1)
x4 = copy.deepcopy(x)
y4 = copy.deepcopy(y)

##
btch1 = parent_url + '/data_batch_5'
load_cifar10(btch1)
x5 = copy.deepcopy(x)
y5 = copy.deepcopy(y)


##
btch1 = parent_url + '/test_batch'
load_cifar10(btch1)
x_test = copy.deepcopy(x)
y_test = copy.deepcopy(y)


# concatenating
# -------------
x_train = np.concatenate((x1,x2,x3,x4,x5), axis = 0)
y_train = np.concatenate((y1,y2,y3,y4,y5), axis = 0)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# Sanity viewing of images
# ------------------------

# random check
# ------------
randrange = list(np.random.randint(x_test.shape[0], size=(1, 2))[0,:])


for i in randrange:
    
    print('TRAIN:')
    curr_loc = np.argwhere(y_train[i]==1)[0,0]
    curr_label = labels_head[b'label_names'][curr_loc]
    print(curr_label)
    plt.imshow(x_train[i])
    plt.show()
    print('------------------')
    print('TEST:')
    curr_loc = np.argwhere(y_test[i]==1)[0,0]
    curr_label = labels_head[b'label_names'][curr_loc]
    print(curr_label)
    plt.imshow(x_test[i])
    plt.show()
    print('------------------')



In [ ]:
# Reshaping y so that it is channel wise
# --------------------------------------
y_train = y_train.reshape(y_train.shape[0],1,1,y_train.shape[1])
y_test = y_test.reshape(y_test.shape[0],1,1,y_test.shape[1])

print(y_train.shape)
print(y_test.shape)

In [ ]:
# splitting set on local machine
# ------------------------------

if dev_env == 'local':
    
    # not doing that for now
    # ----------------------
    x_train = x_train[0:25]
    y_train = y_train[0:25]
    x_test = x_test[0:5]
    y_test = y_test[0:5]
    


# 2. setting up & training models

In [ ]:
# snippet to work out filter sizes
# --------------------------------
f = 2
s = 2
pad = 0
layers = 5

h = 32 # 255
w = 32 # 255
print('Showing conv down sizes - ')
print('--------------------------')
# showing out sizes after conv
# ----------------------------
for _ in range(layers):   
    h,w = outsize_conv(h,w,f,s,pad)
    print((h,w))
    
h = 1
w = 1
print('\nShowing conv up sizes - ')
print('--------------------------')
# showing out sizes after conv
# ----------------------------
dims = []
for _ in range(layers):   
    h,w = outsize_upconv(h,w,f,s,pad)
    dims.append((h,w))
    print((h,w))


In [ ]:
# Resizing images if required
# ----------------------------
#new_h,new_w = 191,319
#resize_images = False

#if resize_images == True:
#    xtrn_src = resize_all(xtrn_src,new_h,new_w)
#    xtst_src = resize_all(xtst_src,new_h,new_w)
    

# printing shapes for sanity
# --------------------------
#print(xtrn_src.shape)
#print(xtrn_tgt.shape) 

In [ ]:
# Set up train data
# -----------------
xin_train = Variable(setup_image_tensor(x_train)).float()
yout_train = Variable(setup_image_tensor(y_train)).float()

print(xin_train.size())
print(yout_train.size())

In [ ]:
# checking class distribution in the training dataset
# --------------------------------------------------

d = return_class_distribution(yout_train.reshape(yout_train.shape[0],10).numpy())

### 2.2 start of model setup and training

In [ ]:
# Using an FCN AE system
# Training fcn_ae_6_layer_WNET to 65 expoch brings loss to 0.014
# ----------------------


# new model initialisations
# -------------------------
target_activation = 'sigmoid'
no_classes_for_act = 10
#model_classnet = fcn_classifier_cifar10_linlayers(3,no_classes_for_act,target_activation)
#model_classnet.apply(weights_init)
#model_classnet

In [ ]:
# training the model
# ------------------

''' USE -1 AS EPOCHS TO LOAD SAVED MODEL WITHOUT TRAINING '''

# model_train(xin,yin,xval,yval,load_mode,model,epochs,mbsize,loss_mode,flatten,use_cuda,save_state,path)

cn_file_name = 'model_classnet_cifar10_sigmoid_simpler_softmaxinternal.tar'
cn_save_path = save_path + cn_file_name
print(cn_save_path)


model_classnet = model_train(xin_train,yout_train*255,None,None,'from saved',None,-1,5,'mse',use_cuda,True,cn_save_path)

### 2.3 checking accuracy & visualising results

In [ ]:
# Checking accuracy
# -----------------

check_accuracy(x_test,y_test,model_classnet,target_activation)

In [ ]:
# random check on test images
# ---------------------------

# setup test set
# --------------
randrange = list(np.random.randint(x_test.shape[0], size=(1, 2))[0,:])
x_vis = x_test[randrange]

vis_out = chunk_pass(setup_image_tensor(x_vis),model_classnet,False,use_cuda,1)
vis_out = vis_out.view(vis_out.size()[0],-1).cpu().data.numpy()

# correcting output
# -----------------
if target_activation == 'sigmoid':
    vis_out[vis_out < 0.5] = 0
else:
    vis_out = softmax_to_onehot(vis_out)

    
# looping
# --------
for i in range(x_vis.shape[0]):
    
    try:
        curr_loc = np.argwhere(vis_out[i]==1)[0,0]
        curr_label = labels_head[b'label_names'][curr_loc]
        print(curr_label)
    except:
        print('No confident prediction')
    
    plt.imshow(x_vis[i])
    plt.show()
    print('------------------')


    
    

### 3. sell all code run

In [ ]:
# getting in images from a folder -- images can be of arbbitary size
# -------------------------------------------------------------------

in_folder = '/Users/venkateshmadhava/Desktop/test'
folder_h,folder_w = 500,500

# using dict
# ----------
#imgs_dict = create_dataset_from_folder_to_dict(in_folder)

# normalising to a single size
# ----------------------------
x_folder = create_dataset_from_folder_all(in_folder,folder_h,folder_w)
print(x_folder.shape)

In [ ]:
# sanity showing of image
# -----------------------

plt.imshow(x_folder[0])

In [ ]:
# setting up see_all models
# -------------------------

out,d = see_all('single',model_classnet.eval(),x_folder,target_activation)

In [ ]:
# checking dims for sanity
# ------------------------

#for keys in d:
#    print(str(keys) + ' f,s: ' + str(d[keys]['f'][0]) + ',' + str(d[keys]['s_list'][0]) + ' | dims: ' + str(d[keys]['new_dims']))

In [ ]:
# displaying see all result
# -------------------------

for i in range(out.shape[0]):
    
    print('At image ' + str(i) + '..')
    
    for c in range(out.shape[3]):
        print('label channel: ' + str(labels_head[b'label_names'][c]))
        plt.imshow(x_folder[i])
        plt.show()
        plt.imshow(out[i,:,:,c])
        plt.show()
        
    print('**************************************************')
    
    
    

In [ ]:
# END OF ALL CODES

# Rough

## rough but keep